In [1]:
import pandas as pd
from snowflake.snowpark import Session
from snowflake.snowpark.functions import *
import matplotlib.pyplot as plt

In [2]:
connection_parameters = {
    "account": "HomeAdvisor",
    "user": "zakir.pasha@HOMEADVISOR.COM",
    "authenticator": "externalbrowser",
    "role": "PROD_MGT",
    "warehouse": "PROD_MGT_FASTER",
    "database": "RPT",
    "schema": "REPORTS"
}

session = Session.builder.configs(connection_parameters).create()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://angi.okta.com/app/angi_snowflakehomeadvisor_1/exktxuc62j6A1SLZT5d6/sso/saml?SAMLRequest=jZJBb%2BIwEIX%2FSuQ9J05SSosFVFmgWyQoEaRl1UvlTQwYYjvrcUjaX79OKKv20Ko3a%2Fye%2Fc286d%2FUIneOTANXcoACz0cOk6nKuNwO0ENy614jBwyVGc2VZAP0wgDdDPtARV6QqDQ7uWR%2FSwbGsQ9JIO3FAJVaEkWBA5FUMCAmJatoPiOh55NCK6NSlaN3lq8dFIBpYwnPlgy4xdsZUxCMq6ryqgtP6S0Ofd%2FHfg9bVSP5cdbXtqdP9AH2O43eKqw8fmP7yeVpBF9h%2FTmJgNwlSezGi1WCnOiMOlISSsH0iukjT9nDcnYCAEtwt5hPovHjdLVYeiBVtcnpgaVKFKWx73n2hDcsw7nactvydDxAxYFnEzMW%2B7DLX%2FeF%2BA3%2Blu6kWM9TGN1HySxe13EBVxf0l6jWkxQ5j%2BdMwybTKUDJprJJ0tiSH166ftcNr5IgIJ0eCa69oNd5Qs7YJsklNa3zjEvllnvqYGhLRouirTz%2FJ98pwWh25KD0c4BZfTB1mXbDfTcKVrOn5DLrYgCFm5zRaXVIy6OH3x9IH7%2F3ve3fvY1kOo5VztMX51ZpQc3niQVe0FZ45m5aKWGC8jzKMs0AbHJ5rqqRZtTYNTe6ZAgPT79%2BXPT

In [3]:
def query(q):
    df = session.sql(q).to_pandas()
    return df

<h1> Count Check </h1>

The purpose of this cell is to create CTEs for all the DBT models for f_presentation, join them together and sample a few days. This will then get compared to the real f_presentation to ensure sp_id counts are similiar across those days.

In [30]:
q = '''

with request_extension as (
SELECT
        oid AS sr_id,
        submitted_date AS sr_submitted_dttm,
        cast(trunc(submitted_date, 'DAY') AS TIMESTAMP_NTZ(9)) AS sr_submitted_date,
        marketplace_id,
        net_sr,
        alternate_source,
        matched_datetime,
        request_status,
        filter_status,
        match_type_id
    FROM RPT.TEMP.WK_BV_WS_SERVICE_REQUESTS_EXTENSION
    WHERE
        1 = 1
        AND test_sr = 'N'
        AND deleted = 0
        --Grabbing at least 2.5 years of data for comprehensive coverage of all pros
        AND modify_datetime >= '2025-06-01'
),

al_cm as (
SELECT
        cast(al.sr_al_pro_id AS STRING) || 'al_cm' || cast(al.al_pro_id AS STRING) AS presentation_id,
        'AL CM' AS presentation_type,  -- angies list combined match/project one
        al.sr_oid AS sr_id,
        al.al_pro_id AS sp_id, --equivalent to a sp_id
        al.create_datetime
    FROM landing.wisen_data.sm_sr_al_pro al
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        al.create_datetime >= '2025-06-01'
        and 
        al_pro_status_id IN (8, 9)
        AND NOT EXISTS (  -- <--- NOT EXISTS clause for 'AL CM'
            SELECT 1
            FROM rpt.intermediate.int_pro_selection_presented fep
            WHERE
                fep.sr_id = al.sr_oid
                AND fep.sp_id = al.al_pro_id
                AND fep.presentation_type = 'AL CM'
        )
    QUALIFY row_number() OVER (
        PARTITION BY al.sr_al_pro_id
        ORDER BY al.create_datetime
    ) = 1
),

al_dbs_h as (
SELECT
        cast(spr.sr_presentation_result_id AS STRING) || 'al_dbs_hist' || cast(spr.sp_id AS STRING) AS presentation_id,
        'AL DBS Historical' AS presentation_type,
        ssp.sr_oid AS sr_id,
        spr.sp_id,
        ssp.create_datetime
    FROM landing.wisen_data.sm_sr_presentation ssp
    INNER JOIN landing.wisen_data.sm_sr_presentation_result spr ON ssp.sr_presentation_id = spr.sr_presentation_id
    INNER JOIN request_extension fsr ON ssp.sr_oid = fsr.sr_id
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        ssp.create_datetime >= '2025-06-01'
        and 
        fsr.match_type_id IN (43, 9, 1, 3, 10, 51, 52, 46, 48)
        AND spr.marketplace_id = 2
        AND spr.sr_prsntn_rslt_type_id IN (14)
        ---and spr.is_phone_displayed is not null
        AND (fsr.net_sr = 1 OR fsr.request_status = 'EXT_FULFIL')
),

al_dbs as (
SELECT
        cast(ap.sr_al_pro_id AS STRING) || 'al_dbs' || cast(ap.al_pro_id AS STRING) AS presentation_id,
        'AL DBS' AS presentation_type,
        ap.sr_oid AS sr_id,
        ap.al_pro_id AS sp_id,
        ap.create_datetime
    FROM landing.wisen_data.sm_sr_al_pro ap
    WHERE --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        ap.create_datetime >= '2025-06-01'
        and ap.al_pro_status_id IN (2, 3, 4)
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY ap.create_datetime
        ) = 1
),

al_tl as (
SELECT
        cast(ap.sr_al_pro_id AS STRING) || 'al_tl' || cast(ap.al_pro_id AS STRING) AS presentation_id,
        'AL TL' AS presentation_type,
        ap.sr_oid AS sr_id,
        ap.al_pro_id AS sp_id,
        ap.create_datetime
    FROM landing.wisen_data.sm_sr_al_pro ap
    WHERE --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        ap.create_datetime >= '2025-06-01'
        and 
        ap.al_pro_status_id IN (5, 6)
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY ap.create_datetime
        ) = 1
),

ald as (
SELECT
        cast(spr.sr_presentation_result_id AS STRING) || 'ald' || cast(spr.sp_id AS STRING) AS presentation_id,
        'ALD' AS presentation_type,
        ssp.sr_oid AS sr_id,
        spr.sp_id,
        ssp.create_datetime
    FROM landing.wisen_data.sm_sr_presentation ssp
    INNER JOIN landing.wisen_data.sm_sr_presentation_result spr ON ssp.sr_presentation_id = spr.sr_presentation_id
    INNER JOIN request_extension fsr ON ssp.sr_oid = fsr.sr_id
    LEFT OUTER JOIN landing.wisen_data.sm_sr_al_pro al ON ssp.sr_oid = al.sr_oid
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        ssp.create_datetime >= '2025-06-01'
        and 
        fsr.match_type_id IN (43, 9, 1, 3, 10, 51, 52, 46, 48)
        AND spr.marketplace_id = 2
        AND spr.sr_prsntn_rslt_type_id NOT IN (14, 16)
        --and spr.is_phone_displayed is not null
        AND (fsr.net_sr = 1 OR fsr.request_status = 'EXT_FULFIL')
        AND al.sr_oid IS NULL
),

basecc as (
        SELECT
        choice_pot_lead_id,
        sr_oid as sr_id,
        sp_id,
        min(create_datetime) AS min_date
    FROM
        landing.wisen_data.sm_choice_pot_lead_hist
    WHERE
        choice_pot_lead_status_id IN (2, 4)
        AND post_submit_match_fg = 'N'
        AND presented_datetime IS NOT NULL
        AND --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        create_datetime >= '2025-06-01'
    GROUP BY choice_pot_lead_id, sr_oid, sp_id
    ),
        
cc as (
SELECT
        cast(cc.choice_pot_lead_id AS STRING) || 'cc' || cast(cc.sp_id AS STRING) AS presentation_id,
        'Consumer Choice' AS presentation_type,
        cc.sr_id,
        cc.sp_id,
        cc.min_date AS create_datetime
    FROM basecc cc
    INNER JOIN request_extension fsr ON cc.sr_id = fsr.sr_id
    WHERE
        (fsr.net_sr = 1 OR fsr.request_status = 'EXT_FULFIL')
        AND cast(fsr.sr_submitted_dttm AS DATE) BETWEEN cast('2019-09-30' AS DATE) AND current_timestamp
        AND NOT EXISTS (  -- <--- NOT EXISTS clause for 'Consumer Choice'
            SELECT 1
            FROM rpt.intermediate.int_pro_selection_presented fep
            WHERE
                fep.sr_id = cc.sr_id
                AND fep.sp_id = cc.sp_id
                AND fep.presentation_type = 'Consumer Choice'
        )

    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY cc.min_date
        ) = 1
),

cj as (
SELECT
        cast(cj.sr_cj_pro_id AS STRING) || 'cj' || cast(cj.sp_id AS STRING) AS presentation_id,
        'CJ' AS presentation_type,  -- angies list combined match/project one
        cj.sr_oid AS sr_id,
        cj.sp_id, --equivalent to a sp_id
        cj.create_datetime
    FROM landing.wisen_data.SM_SR_CJ_PRO cj
    WHERE --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        cj.create_datetime >= '2025-06-01'
        and 
        cj_pro_status_id IN (1, 2)
),

srs_with_ol_pros_presented AS (
    SELECT sr_oid as sr_id
    FROM rpt.intermediate.int_ol

    UNION DISTINCT

    SELECT sr_id
    FROM rpt.intermediate.int_pro_selection_presented
),

fpp as (
select 
        cast(eft.ext_fulfillment_trans_id AS STRING) || 'fpp' || '0' AS presentation_id,
        'Fixed Price Path' AS presentation_type,
        eft.sr_oid AS sr_id,
        NULL AS sp_id,
        eft.create_datetime
    FROM landing.wisen_data.SM_EXT_FULFILLMENT_TRANS eft
    INNER JOIN request_extension fsr
    ON eft.sr_oid = fsr.sr_id
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        eft.create_datetime >= '2025-06-01'
        and 
        (
        --2025-06-11 is the first full day when FPP traffic was migrated to 
        -- the new frontend in next-sr-path. A Service Request that happend 
        -- after this date and has all of: 
        --  1. An OL presentation 
        --  2. Match type 56 
        --  3. Alternate source SITE or AL_SITE
        --  Should not be considered an FPP presentation.
            eft.create_datetime < '2025-06-11'
            OR
            (
                eft.create_datetime >= '2025-06-11'
                AND (
                    fsr.match_type_id NOT IN (56)
                    OR fsr.alternate_source NOT IN (1, 14)
                    OR NOT EXISTS (
                        SELECT 1
                        FROM srs_with_ol_pros_presented olp
                        WHERE olp.sr_id = eft.sr_oid
                    )
                )
            )
        )
    QUALIFY
        row_number() OVER (
            PARTITION BY eft.sr_oid
            ORDER BY eft.create_datetime
        ) = 1
),

ib as (
SELECT
        cast(ib.ob_potential_lead_id AS STRING) || 'ib' || cast(ib.sp_id AS STRING) AS presentation_id,
        'Instant Booking' AS presentation_type,
        ib.sr_oid AS sr_id,
        ib.sp_id,
        ib.create_datetime
    FROM landing.wisen_data.SM_OB_POTENTIAL_LEAD ib
    INNER JOIN request_extension fsr ON ib.sr_oid = fsr.sr_id
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        ib.create_datetime >= '2025-06-01'
        and 
        fsr.match_type_id IN (25, 26, 56)
        AND ib.display_seq > 0
        AND NOT EXISTS (  -- <--- NOT EXISTS clause for 'Instant Booking'
            SELECT 1
            FROM rpt.intermediate.int_pro_selection_presented fep
            WHERE
                fep.sr_id = ib.sr_oid
                AND fep.sp_id = ib.sp_id
                AND fep.presentation_type = 'Instant Booking'
        )
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY ib.create_datetime
        ) = 1
),

ic as (
SELECT
        cast(ic.ic_pot_lead_id AS STRING) || 'ic' || cast(ic.sp_id AS STRING) AS presentation_id,
        'Instant Connect' AS presentation_type,
        ic.sr_oid AS sr_id,
        ic.sp_id,
        ic.create_datetime
    FROM landing.wisen_data.SM_INSTANT_CONNECT_POT_LEAD ic
    WHERE --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        ic.create_datetime >= '2025-06-01'
        and 
        ic.status_id IN (3) --removed 1 which is 'not called'. including 1 returned the pool of SPs, not just the one connected
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY ic.create_datetime
        ) = 1
),

mhc as (
SELECT
        cast(poh.prepriced_order_id AS STRING) || 'mhc' || '0' AS presentation_id,
        'MHC' AS presentation_type,
        poh.sr_oid AS sr_id,
        NULL AS sp_id,
        poh.create_datetime
    FROM landing.product_storefront.PREPRICED_ORDER_HIST poh
    WHERE 
    --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        poh.create_datetime >= '2025-06-01'
        and poh.sr_oid IS NOT NULL
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY poh.create_datetime
        ) = 1
),

mmr as (
SELECT
        cast(pm.map_id AS STRING) || 'mmr' || cast(pm.sp_id AS STRING) AS presentation_id,
        'MMR' AS presentation_type,
        pm.sr_oid AS sr_id,
        pm.sp_id,
        pm.create_datetime
    FROM landing.wisen_data.SM_XM_DIR_PHONE_MAP pm
    INNER JOIN request_extension fsr ON pm.sr_oid = fsr.sr_id
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        pm.create_datetime >= '2025-06-01'
        and 
        match_type_id IN (9, 10, 24, 56)
        AND (fsr.net_sr = 1 OR fsr.request_status = 'EXT_FULFIL')
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY pm.create_datetime
        ) = 1
),

mw as (
SELECT
        cast(fl.lead_id AS STRING) || 'mm' || cast(fl.sp_id AS STRING) AS presentation_id,
        'Market Match' AS presentation_type,
        fl.sr_oid as sr_id,
        fl.sp_id,
        fl.create_datetime
    FROM landing.wisen_data.sm_lead fl
    INNER JOIN REQUEST_EXTENSION fsr ON fl.sr_oid = fsr.sr_id
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        fl.create_datetime >= '2025-06-01'
        and 
        (fsr.net_sr = 1 OR fsr.request_status = 'EXT_FULFIL')
        AND fsr.match_type_id IN (1, 43)
        AND fl.lead_type_id IN (2)
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY fl.create_datetime
        ) = 1
),

mmr as (
SELECT
        cast(fl.lead_id AS STRING) || 'mm' || cast(fl.sp_id AS STRING) AS presentation_id,
        'Market Match' AS presentation_type,
        fl.sr_oid,
        fl.sp_id,
        fl.create_datetime
    FROM landing.wisen_data.sm_lead fl
    INNER JOIN request_extension fsr ON fl.sr_oid = fsr.sr_id
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        fl.create_datetime >= '2025-06-01'
        and 
        (fsr.net_sr = 1 OR fsr.request_status = 'EXT_FULFIL')
        AND fsr.match_type_id IN (1, 43)
        AND fl.lead_type_id IN (2)
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY fl.create_datetime
        ) = 1
),

ol_tl as (
SELECT
        cast(fol.ol_pot_lead_id AS STRING) || 'oltl' || cast(fol.sp_id AS STRING) AS presentation_id,
        'OL TL' AS presentation_type,
        fol.sr_oid as sr_id,
        fol.sp_id,
        fol.create_datetime
    FROM rpt.intermediate.int_ol fol
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        fol.create_datetime >= '2025-06-01'
        and 
        fol.ol_type_id != 2
        AND fol.ol_type_id != 3
        AND fol.sp_presented_on_tl = 1
        AND NOT EXISTS (  -- <--- NOT EXISTS clause for 'OL TL'
            SELECT 1
            FROM rpt.intermediate.int_pro_selection_presented fep
            WHERE
                fep.sr_id = fol.sr_oid
                AND fep.sp_id = fol.sp_id
                AND fep.presentation_type = 'OL TL'
        )
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY fol.create_datetime
        ) = 1
),

pf as (
SELECT
        cast(ppl.pmp_pot_lead_id AS STRING) || 'profinder' || cast(ppl.sp_id AS STRING) AS presentation_id,
        'Profinder' AS presentation_type,
        ppl.sr_oid AS sr_id,
        ppl.sp_id,
        ppl.create_datetime
    FROM landing.wisen_data.SM_PMP_POT_LEAD ppl
    WHERE --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        ppl.create_datetime >= '2025-06-01'
        and 
        pmp_pot_lead_status_id IN (2, 4)
),

ping_post as (
SELECT
        bid_pro.bidproid,
        bid_pro.id_bidkey,
        bid_pro.id_legacyserviceproviderid,
        bid_service_request.servicerequestid,
        bid_pro.createtime_ntz,
        bid_pro.met
    FROM
        landing.kafka.ppp_data_bid_pro AS bid_pro
    INNER JOIN landing.kafka.PPP_DATA_BID_SERVICE_REQUEST AS bid_service_request
    ON
        bid_pro.id_bidkey = bid_service_request.bidkey
    WHERE
        bid_pro.met in ('XML_OL','XML_SR_BID')
    and 
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        bid_pro.createtime_ntz >= '2025-06-01'
),

ppp_ol as (
    SELECT
        bidproid || 'pppol' || id_legacyserviceproviderid AS presentation_id,
        'ping-post-pick OL' AS presentation_type,
        servicerequestid AS sr_id,
        id_legacyserviceproviderid AS sp_id,
        createtime_ntz AS create_datetime
    FROM
        ping_post AS bid_pro
    WHERE 
        met = 'XML_OL'
    QUALIFY
        row_number() OVER (
            PARTITION BY 
                CAST(bidproid AS VARCHAR) || 'pppol' || CAST(id_legacyserviceproviderid AS VARCHAR)
            ORDER BY createtime_ntz DESC
        ) = 1
),

ppp as (
SELECT
        bidproid || 'ppp' || id_legacyserviceproviderid AS presentation_id,
        'ping-post-pick' AS presentation_type,
        servicerequestid AS sr_id,
        id_legacyserviceproviderid AS sp_id,
        createtime_ntz AS create_datetime
    FROM
        ping_post
    WHERE
        met = 'XML_SR_BID'
    QUALIFY
        row_number() OVER (
            PARTITION BY bidproid || 'ppp' || id_legacyserviceproviderid
            ORDER BY createtime_ntz DESC
        ) = 1
),

psm as (
SELECT
        cast(cplh.choice_pot_lead_id AS STRING) || 'psm' || cast(cplh.sp_id AS STRING) AS presentation_id,
        'PSM' AS presentation_type,
        cplh.sr_oid AS sr_id,
        cplh.sp_id,
        cplh.create_datetime
    FROM landing.wisen_data.SM_CHOICE_POT_LEAD_HIST cplh
    INNER JOIN request_extension fsr ON cplh.sr_oid = fsr.sr_id
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        cplh.create_datetime >= '2025-06-01'
        and 
        cplh.post_submit_match_fg = 'Y'
        AND choice_pot_lead_status_id IN (2)
        AND cplh.presented_datetime IS NOT NULL
        AND (fsr.net_sr = 1 OR fsr.request_status = 'EXT_FULFIL')
        AND fsr.sr_submitted_dttm < current_timestamp
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY cplh.create_datetime
        ) = 1
),

sd as (
SELECT
        cast(sdb.sdb_potential_lead_id AS STRING) || 'sdb' || cast(sdb.sp_id AS STRING) AS presentation_id,
        'Same Day Booking' AS presentation_type,
        sdb.sr_oid AS sr_id,
        sdb.sp_id,
        sdb.create_datetime
    FROM landing.wisen_data.SM_SDB_POTENTIAL_LEAD sdb
    where
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        sdb.create_datetime >= '2025-06-01'
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY sdb.create_datetime
        ) = 1
),

spi as (
SELECT
        cast(fol.ol_pot_lead_id AS STRING) || 'spi' || cast(fol.sp_id AS STRING) AS presentation_id,
        'SP Invite' AS presentation_type,
        fol.sr_oid as sr_id,
        fol.sp_id,
        fol.create_datetime
    FROM rpt.intermediate.int_ol fol
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        fol.create_datetime >= '2025-06-01'
        and
        fol.sp_invites = 1
        AND fol.sp_presented_on_tl = 0
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY fol.create_datetime
        ) = 1
),

wj as (
SELECT
        cast(fol.ol_pot_lead_id AS STRING) || 'winjobs' || cast(fol.sp_id AS STRING) AS presentation_id,
        'Win Jobs' AS presentation_type,
        fol.sr_oid as sr_id,
        fol.sp_id,
        fol.create_datetime
    FROM rpt.intermediate.int_ol fol
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        fol.create_datetime >= '2025-06-01'
        AND 
            fol.ol_type_id = 3
        AND fol.sp_presented_on_tl = 1
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY fol.create_datetime
        ) = 1
),

final_prez as (
SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        datetime_mst AS create_datetime
    FROM rpt.intermediate.int_pro_selection_presented
    WHERE 
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        datetime_mst >= '2025-06-01'

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM cc

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM ib

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM ic

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM fpp

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM mmr

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM ppp

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM ppp_ol

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM sd

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM mhc

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM ol_tl

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM spi

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM wj


    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM psm

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM ald

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM al_tl

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM al_dbs

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM al_dbs_h

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM mw

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM pf

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM cj

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM al_cm
),

base_migrated as (
    select 
        sr_id,
        sp_id,
        presentation_type,
        create_datetime 
    from 
        final_prez
    where 
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        create_datetime >= '2025-06-01'
    ),
    
    double_counts_ib_cc AS (
    SELECT
        sr_id,
        sp_id,
        'Instant Booking' AS presentation_type
    FROM base_migrated
    WHERE presentation_type IN ('Consumer Choice', 'Instant Booking')
    GROUP BY 1, 2
    HAVING count(*) > 1
    ),
    
Z AS (
SELECT
    b.sp_id,
    b.create_datetime,
    fsr.sr_submitted_date,
    fsr.matched_datetime AS sr_matched_datetime,
    b.presentation_type
FROM base_migrated b
INNER JOIN request_extension  fsr ON b.sr_id = fsr.sr_id
LEFT OUTER JOIN landing.wisen_data.sm_lead sl ON b.sr_id = sl.sr_oid AND b.sp_id = sl.sp_id
LEFT OUTER JOIN double_counts_ib_cc db ON (b.sr_id = db.sr_id AND b.sp_id = db.sp_id)
WHERE
    (sl.lead_type_id != 9 OR sl.lead_type_id IS NULL)
    AND (db.presentation_type IS NULL OR db.presentation_type = b.presentation_type)
),

z_final as (
select 
    date(create_datetime) as date,
    count(distinct sp_id) as pros 
from 
    Z 
group by 1),

real as (
select 
    date(create_datetime) as date,
    count(distinct sp_id) as pros 
from 
    rpt.reports.f_presentation
where 
    create_datetime >= '2025-06-01'
group by 1)

select 
    a.date,
    a.pros as zakir_pros,
    b.pros as real_pros,
    (a.pros/b.pros) *100
from 
    z_final a 
left join 
    real b 
on a.date = b.date
order by 1


'''

df = query(q)

In [31]:
df

,DATE,ZAKIR_PROS,REAL_PROS,(A.PROS/B.PROS) *100
0,2025-06-01,51647,51701,99.8956
1,2025-06-02,64828,64909,99.8752
2,2025-06-03,62291,62333,99.9326
3,2025-06-04,62779,62814,99.9443
4,2025-06-05,61941,61975,99.9451
5,2025-06-06,59108,59109,99.9983
6,2025-06-07,51904,51912,99.9846
7,2025-06-08,50628,50635,99.9862
8,2025-06-09,63145,63150,99.9921
9,2025-06-10,60813,60817,99.9934


<h1> Lead Pro Active Check </h1>

The purpose of this cell is to take the logic above, implement it into RPT.INTERMEDIATE.INT_LEADS_ACTIVE_RANGE and ensure it lines up with the expected values.

In [32]:
q = '''

with request_extension as (
SELECT
        oid AS sr_id,
        submitted_date AS sr_submitted_dttm,
        cast(trunc(submitted_date, 'DAY') AS TIMESTAMP_NTZ(9)) AS sr_submitted_date,
        marketplace_id,
        net_sr,
        alternate_source,
        matched_datetime,
        request_status,
        filter_status,
        match_type_id
    FROM RPT.TEMP.WK_BV_WS_SERVICE_REQUESTS_EXTENSION
    WHERE
        1 = 1
        AND test_sr = 'N'
        AND deleted = 0
        --Grabbing at least 2.5 years of data for comprehensive coverage of all pros
        AND modify_datetime >= '2023-01-01'
),

al_cm as (
SELECT
        cast(al.sr_al_pro_id AS STRING) || 'al_cm' || cast(al.al_pro_id AS STRING) AS presentation_id,
        'AL CM' AS presentation_type,  -- angies list combined match/project one
        al.sr_oid AS sr_id,
        al.al_pro_id AS sp_id, --equivalent to a sp_id
        al.create_datetime
    FROM landing.wisen_data.sm_sr_al_pro al
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        al.create_datetime >= '2023-01-01'
        and 
        al_pro_status_id IN (8, 9)
        AND NOT EXISTS (  -- <--- NOT EXISTS clause for 'AL CM'
            SELECT 1
            FROM rpt.intermediate.int_pro_selection_presented fep
            WHERE
                fep.sr_id = al.sr_oid
                AND fep.sp_id = al.al_pro_id
                AND fep.presentation_type = 'AL CM'
        )
    QUALIFY row_number() OVER (
        PARTITION BY al.sr_al_pro_id
        ORDER BY al.create_datetime
    ) = 1
),

al_dbs_h as (
SELECT
        cast(spr.sr_presentation_result_id AS STRING) || 'al_dbs_hist' || cast(spr.sp_id AS STRING) AS presentation_id,
        'AL DBS Historical' AS presentation_type,
        ssp.sr_oid AS sr_id,
        spr.sp_id,
        ssp.create_datetime
    FROM landing.wisen_data.sm_sr_presentation ssp
    INNER JOIN landing.wisen_data.sm_sr_presentation_result spr ON ssp.sr_presentation_id = spr.sr_presentation_id
    INNER JOIN request_extension fsr ON ssp.sr_oid = fsr.sr_id
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        ssp.create_datetime >= '2023-01-01'
        and 
        fsr.match_type_id IN (43, 9, 1, 3, 10, 51, 52, 46, 48)
        AND spr.marketplace_id = 2
        AND spr.sr_prsntn_rslt_type_id IN (14)
        ---and spr.is_phone_displayed is not null
        AND (fsr.net_sr = 1 OR fsr.request_status = 'EXT_FULFIL')
),

al_dbs as (
SELECT
        cast(ap.sr_al_pro_id AS STRING) || 'al_dbs' || cast(ap.al_pro_id AS STRING) AS presentation_id,
        'AL DBS' AS presentation_type,
        ap.sr_oid AS sr_id,
        ap.al_pro_id AS sp_id,
        ap.create_datetime
    FROM landing.wisen_data.sm_sr_al_pro ap
    WHERE --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        ap.create_datetime >= '2023-01-01'
        and ap.al_pro_status_id IN (2, 3, 4)
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY ap.create_datetime
        ) = 1
),

al_tl as (
SELECT
        cast(ap.sr_al_pro_id AS STRING) || 'al_tl' || cast(ap.al_pro_id AS STRING) AS presentation_id,
        'AL TL' AS presentation_type,
        ap.sr_oid AS sr_id,
        ap.al_pro_id AS sp_id,
        ap.create_datetime
    FROM landing.wisen_data.sm_sr_al_pro ap
    WHERE --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        ap.create_datetime >= '2023-01-01'
        and 
        ap.al_pro_status_id IN (5, 6)
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY ap.create_datetime
        ) = 1
),

ald as (
SELECT
        cast(spr.sr_presentation_result_id AS STRING) || 'ald' || cast(spr.sp_id AS STRING) AS presentation_id,
        'ALD' AS presentation_type,
        ssp.sr_oid AS sr_id,
        spr.sp_id,
        ssp.create_datetime
    FROM landing.wisen_data.sm_sr_presentation ssp
    INNER JOIN landing.wisen_data.sm_sr_presentation_result spr ON ssp.sr_presentation_id = spr.sr_presentation_id
    INNER JOIN request_extension fsr ON ssp.sr_oid = fsr.sr_id
    LEFT OUTER JOIN landing.wisen_data.sm_sr_al_pro al ON ssp.sr_oid = al.sr_oid
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        ssp.create_datetime >= '2023-01-01'
        and 
        fsr.match_type_id IN (43, 9, 1, 3, 10, 51, 52, 46, 48)
        AND spr.marketplace_id = 2
        AND spr.sr_prsntn_rslt_type_id NOT IN (14, 16)
        --and spr.is_phone_displayed is not null
        AND (fsr.net_sr = 1 OR fsr.request_status = 'EXT_FULFIL')
        AND al.sr_oid IS NULL
),

basecc as (
        SELECT
        choice_pot_lead_id,
        sr_oid as sr_id,
        sp_id,
        min(create_datetime) AS min_date
    FROM
        landing.wisen_data.sm_choice_pot_lead_hist
    WHERE
        choice_pot_lead_status_id IN (2, 4)
        AND post_submit_match_fg = 'N'
        AND presented_datetime IS NOT NULL
        AND --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        create_datetime >= '2023-01-01'
    GROUP BY choice_pot_lead_id, sr_oid, sp_id
    ),
        
cc as (
SELECT
        cast(cc.choice_pot_lead_id AS STRING) || 'cc' || cast(cc.sp_id AS STRING) AS presentation_id,
        'Consumer Choice' AS presentation_type,
        cc.sr_id,
        cc.sp_id,
        cc.min_date AS create_datetime
    FROM basecc cc
    INNER JOIN request_extension fsr ON cc.sr_id = fsr.sr_id
    WHERE
        (fsr.net_sr = 1 OR fsr.request_status = 'EXT_FULFIL')
        AND cast(fsr.sr_submitted_dttm AS DATE) BETWEEN cast('2019-09-30' AS DATE) AND current_timestamp
        AND NOT EXISTS (  -- <--- NOT EXISTS clause for 'Consumer Choice'
            SELECT 1
            FROM rpt.intermediate.int_pro_selection_presented fep
            WHERE
                fep.sr_id = cc.sr_id
                AND fep.sp_id = cc.sp_id
                AND fep.presentation_type = 'Consumer Choice'
        )

    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY cc.min_date
        ) = 1
),

cj as (
SELECT
        cast(cj.sr_cj_pro_id AS STRING) || 'cj' || cast(cj.sp_id AS STRING) AS presentation_id,
        'CJ' AS presentation_type,  -- angies list combined match/project one
        cj.sr_oid AS sr_id,
        cj.sp_id, --equivalent to a sp_id
        cj.create_datetime
    FROM landing.wisen_data.SM_SR_CJ_PRO cj
    WHERE --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        cj.create_datetime >= '2023-01-01'
        and 
        cj_pro_status_id IN (1, 2)
),

srs_with_ol_pros_presented AS (
    SELECT sr_oid as sr_id
    FROM rpt.intermediate.int_ol

    UNION DISTINCT

    SELECT sr_id
    FROM rpt.intermediate.int_pro_selection_presented
),

fpp as (
select 
        cast(eft.ext_fulfillment_trans_id AS STRING) || 'fpp' || '0' AS presentation_id,
        'Fixed Price Path' AS presentation_type,
        eft.sr_oid AS sr_id,
        NULL AS sp_id,
        eft.create_datetime
    FROM landing.wisen_data.SM_EXT_FULFILLMENT_TRANS eft
    INNER JOIN request_extension fsr
    ON eft.sr_oid = fsr.sr_id
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        eft.create_datetime >= '2023-01-01'
        and 
        (
        --2025-06-11 is the first full day when FPP traffic was migrated to 
        -- the new frontend in next-sr-path. A Service Request that happend 
        -- after this date and has all of: 
        --  1. An OL presentation 
        --  2. Match type 56 
        --  3. Alternate source SITE or AL_SITE
        --  Should not be considered an FPP presentation.
            eft.create_datetime < '2025-06-11'
            OR
            (
                eft.create_datetime >= '2025-06-11'
                AND (
                    fsr.match_type_id NOT IN (56)
                    OR fsr.alternate_source NOT IN (1, 14)
                    OR NOT EXISTS (
                        SELECT 1
                        FROM srs_with_ol_pros_presented olp
                        WHERE olp.sr_id = eft.sr_oid
                    )
                )
            )
        )
    QUALIFY
        row_number() OVER (
            PARTITION BY eft.sr_oid
            ORDER BY eft.create_datetime
        ) = 1
),

ib as (
SELECT
        cast(ib.ob_potential_lead_id AS STRING) || 'ib' || cast(ib.sp_id AS STRING) AS presentation_id,
        'Instant Booking' AS presentation_type,
        ib.sr_oid AS sr_id,
        ib.sp_id,
        ib.create_datetime
    FROM landing.wisen_data.SM_OB_POTENTIAL_LEAD ib
    INNER JOIN request_extension fsr ON ib.sr_oid = fsr.sr_id
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        ib.create_datetime >= '2023-01-01'
        and 
        fsr.match_type_id IN (25, 26, 56)
        AND ib.display_seq > 0
        AND NOT EXISTS (  -- <--- NOT EXISTS clause for 'Instant Booking'
            SELECT 1
            FROM rpt.intermediate.int_pro_selection_presented fep
            WHERE
                fep.sr_id = ib.sr_oid
                AND fep.sp_id = ib.sp_id
                AND fep.presentation_type = 'Instant Booking'
        )
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY ib.create_datetime
        ) = 1
),

ic as (
SELECT
        cast(ic.ic_pot_lead_id AS STRING) || 'ic' || cast(ic.sp_id AS STRING) AS presentation_id,
        'Instant Connect' AS presentation_type,
        ic.sr_oid AS sr_id,
        ic.sp_id,
        ic.create_datetime
    FROM landing.wisen_data.SM_INSTANT_CONNECT_POT_LEAD ic
    WHERE --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        ic.create_datetime >= '2023-01-01'
        and 
        ic.status_id IN (3) --removed 1 which is 'not called'. including 1 returned the pool of SPs, not just the one connected
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY ic.create_datetime
        ) = 1
),

mhc as (
SELECT
        cast(poh.prepriced_order_id AS STRING) || 'mhc' || '0' AS presentation_id,
        'MHC' AS presentation_type,
        poh.sr_oid AS sr_id,
        NULL AS sp_id,
        poh.create_datetime
    FROM landing.product_storefront.PREPRICED_ORDER_HIST poh
    WHERE 
    --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        poh.create_datetime >= '2023-01-01'
        and poh.sr_oid IS NOT NULL
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY poh.create_datetime
        ) = 1
),

mmr as (
SELECT
        cast(pm.map_id AS STRING) || 'mmr' || cast(pm.sp_id AS STRING) AS presentation_id,
        'MMR' AS presentation_type,
        pm.sr_oid AS sr_id,
        pm.sp_id,
        pm.create_datetime
    FROM landing.wisen_data.SM_XM_DIR_PHONE_MAP pm
    INNER JOIN request_extension fsr ON pm.sr_oid = fsr.sr_id
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        pm.create_datetime >= '2023-01-01'
        and 
        match_type_id IN (9, 10, 24, 56)
        AND (fsr.net_sr = 1 OR fsr.request_status = 'EXT_FULFIL')
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY pm.create_datetime
        ) = 1
),

mw as (
SELECT
        cast(fl.lead_id AS STRING) || 'mm' || cast(fl.sp_id AS STRING) AS presentation_id,
        'Market Match' AS presentation_type,
        fl.sr_oid as sr_id,
        fl.sp_id,
        fl.create_datetime
    FROM landing.wisen_data.sm_lead fl
    INNER JOIN REQUEST_EXTENSION fsr ON fl.sr_oid = fsr.sr_id
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        fl.create_datetime >= '2023-01-01'
        and 
        (fsr.net_sr = 1 OR fsr.request_status = 'EXT_FULFIL')
        AND fsr.match_type_id IN (1, 43)
        AND fl.lead_type_id IN (2)
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY fl.create_datetime
        ) = 1
),

mmr as (
SELECT
        cast(fl.lead_id AS STRING) || 'mm' || cast(fl.sp_id AS STRING) AS presentation_id,
        'Market Match' AS presentation_type,
        fl.sr_oid,
        fl.sp_id,
        fl.create_datetime
    FROM landing.wisen_data.sm_lead fl
    INNER JOIN request_extension fsr ON fl.sr_oid = fsr.sr_id
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        fl.create_datetime >= '2023-01-01'
        and 
        (fsr.net_sr = 1 OR fsr.request_status = 'EXT_FULFIL')
        AND fsr.match_type_id IN (1, 43)
        AND fl.lead_type_id IN (2)
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY fl.create_datetime
        ) = 1
),

ol_tl as (
SELECT
        cast(fol.ol_pot_lead_id AS STRING) || 'oltl' || cast(fol.sp_id AS STRING) AS presentation_id,
        'OL TL' AS presentation_type,
        fol.sr_oid as sr_id,
        fol.sp_id,
        fol.create_datetime
    FROM rpt.intermediate.int_ol fol
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        fol.create_datetime >= '2023-01-01'
        and 
        fol.ol_type_id != 2
        AND fol.ol_type_id != 3
        AND fol.sp_presented_on_tl = 1
        AND NOT EXISTS (  -- <--- NOT EXISTS clause for 'OL TL'
            SELECT 1
            FROM rpt.intermediate.int_pro_selection_presented fep
            WHERE
                fep.sr_id = fol.sr_oid
                AND fep.sp_id = fol.sp_id
                AND fep.presentation_type = 'OL TL'
        )
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY fol.create_datetime
        ) = 1
),

pf as (
SELECT
        cast(ppl.pmp_pot_lead_id AS STRING) || 'profinder' || cast(ppl.sp_id AS STRING) AS presentation_id,
        'Profinder' AS presentation_type,
        ppl.sr_oid AS sr_id,
        ppl.sp_id,
        ppl.create_datetime
    FROM landing.wisen_data.SM_PMP_POT_LEAD ppl
    WHERE --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        ppl.create_datetime >= '2023-01-01'
        and 
        pmp_pot_lead_status_id IN (2, 4)
),

ping_post as (
SELECT
        bid_pro.bidproid,
        bid_pro.id_bidkey,
        bid_pro.id_legacyserviceproviderid,
        bid_service_request.servicerequestid,
        bid_pro.createtime_ntz,
        bid_pro.met
    FROM
        landing.kafka.ppp_data_bid_pro AS bid_pro
    INNER JOIN landing.kafka.PPP_DATA_BID_SERVICE_REQUEST AS bid_service_request
    ON
        bid_pro.id_bidkey = bid_service_request.bidkey
    WHERE
        bid_pro.met in ('XML_OL','XML_SR_BID')
    and 
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        bid_pro.createtime_ntz >= '2023-01-01'
),

ppp_ol as (
    SELECT
        bidproid || 'pppol' || id_legacyserviceproviderid AS presentation_id,
        'ping-post-pick OL' AS presentation_type,
        servicerequestid AS sr_id,
        id_legacyserviceproviderid AS sp_id,
        createtime_ntz AS create_datetime
    FROM
        ping_post AS bid_pro
    WHERE 
        met = 'XML_OL'
    QUALIFY
        row_number() OVER (
            PARTITION BY 
                CAST(bidproid AS VARCHAR) || 'pppol' || CAST(id_legacyserviceproviderid AS VARCHAR)
            ORDER BY createtime_ntz DESC
        ) = 1
),

ppp as (
SELECT
        bidproid || 'ppp' || id_legacyserviceproviderid AS presentation_id,
        'ping-post-pick' AS presentation_type,
        servicerequestid AS sr_id,
        id_legacyserviceproviderid AS sp_id,
        createtime_ntz AS create_datetime
    FROM
        ping_post
    WHERE
        met = 'XML_SR_BID'
    QUALIFY
        row_number() OVER (
            PARTITION BY bidproid || 'ppp' || id_legacyserviceproviderid
            ORDER BY createtime_ntz DESC
        ) = 1
),

psm as (
SELECT
        cast(cplh.choice_pot_lead_id AS STRING) || 'psm' || cast(cplh.sp_id AS STRING) AS presentation_id,
        'PSM' AS presentation_type,
        cplh.sr_oid AS sr_id,
        cplh.sp_id,
        cplh.create_datetime
    FROM landing.wisen_data.SM_CHOICE_POT_LEAD_HIST cplh
    INNER JOIN request_extension fsr ON cplh.sr_oid = fsr.sr_id
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        cplh.create_datetime >= '2023-01-01'
        and 
        cplh.post_submit_match_fg = 'Y'
        AND choice_pot_lead_status_id IN (2)
        AND cplh.presented_datetime IS NOT NULL
        AND (fsr.net_sr = 1 OR fsr.request_status = 'EXT_FULFIL')
        AND fsr.sr_submitted_dttm < current_timestamp
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY cplh.create_datetime
        ) = 1
),

sd as (
SELECT
        cast(sdb.sdb_potential_lead_id AS STRING) || 'sdb' || cast(sdb.sp_id AS STRING) AS presentation_id,
        'Same Day Booking' AS presentation_type,
        sdb.sr_oid AS sr_id,
        sdb.sp_id,
        sdb.create_datetime
    FROM landing.wisen_data.SM_SDB_POTENTIAL_LEAD sdb
    where
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        sdb.create_datetime >= '2023-01-01'
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY sdb.create_datetime
        ) = 1
),

spi as (
SELECT
        cast(fol.ol_pot_lead_id AS STRING) || 'spi' || cast(fol.sp_id AS STRING) AS presentation_id,
        'SP Invite' AS presentation_type,
        fol.sr_oid as sr_id,
        fol.sp_id,
        fol.create_datetime
    FROM rpt.intermediate.int_ol fol
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        fol.create_datetime >= '2023-01-01'
        and
        fol.sp_invites = 1
        AND fol.sp_presented_on_tl = 0
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY fol.create_datetime
        ) = 1
),

wj as (
SELECT
        cast(fol.ol_pot_lead_id AS STRING) || 'winjobs' || cast(fol.sp_id AS STRING) AS presentation_id,
        'Win Jobs' AS presentation_type,
        fol.sr_oid as sr_id,
        fol.sp_id,
        fol.create_datetime
    FROM rpt.intermediate.int_ol fol
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        fol.create_datetime >= '2023-01-01'
        AND 
            fol.ol_type_id = 3
        AND fol.sp_presented_on_tl = 1
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY fol.create_datetime
        ) = 1
),

final_prez as (
SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        datetime_mst AS create_datetime
    FROM rpt.intermediate.int_pro_selection_presented
    WHERE 
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        datetime_mst >= '2023-01-01'

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM cc

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM ib

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM ic

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM fpp

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM mmr

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM ppp

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM ppp_ol

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM sd

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM mhc

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM ol_tl

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM spi

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM wj


    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM psm

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM ald

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM al_tl

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM al_dbs

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM al_dbs_h

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM mw

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM pf

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM cj

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM al_cm
),

base_migrated as (
    select 
        sr_id,
        sp_id,
        presentation_type,
        create_datetime 
    from 
        final_prez
    where 
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        create_datetime >= '2023-01-01'
    ),
    
    double_counts_ib_cc AS (
    SELECT
        sr_id,
        sp_id,
        'Instant Booking' AS presentation_type
    FROM base_migrated
    WHERE presentation_type IN ('Consumer Choice', 'Instant Booking')
    GROUP BY 1, 2
    HAVING count(*) > 1
    ),
    
Z AS (
SELECT
    b.sp_id,
    b.create_datetime,
    fsr.sr_submitted_date,
    fsr.matched_datetime AS sr_matched_datetime,
    b.presentation_type
FROM base_migrated b
INNER JOIN request_extension  fsr ON b.sr_id = fsr.sr_id
LEFT OUTER JOIN landing.wisen_data.sm_lead sl ON b.sr_id = sl.sr_oid AND b.sp_id = sl.sp_id
LEFT OUTER JOIN double_counts_ib_cc db ON (b.sr_id = db.sr_id AND b.sp_id = db.sp_id)
WHERE
    (sl.lead_type_id != 9 OR sl.lead_type_id IS NULL)
    AND (db.presentation_type IS NULL OR db.presentation_type = b.presentation_type)
),

---------

sp_status_monthly AS (
    SELECT
        sp_id,
        DATE_TRUNC('month', d_date) AS month,
        MAX(CASE WHEN status_code = 3 THEN 1 ELSE 0 END) AS approved_flag
    FROM rpt.temp.wk_sp_status_time_daily 
    WHERE
    --Grabbing at least 2 years of data for comprehensive coverage of all pros
        DATE_TRUNC('month', d_date) >= '2023-01-01'
    GROUP BY sp_id, DATE_TRUNC('month', d_date)
),

active_months_temp AS (
    SELECT
        l.sp_id,
        DATE_TRUNC('month', l.lead_status_datetime) AS month
    FROM landing.wisen_data.sm_lead  AS l
    INNER JOIN landing.wisen_data.bv_ws_service_requests AS sr
    ON l.sr_oid = sr.oid
    WHERE
        l.lead_status = 2
        AND sr.test_sr = 'N'
        AND l.lead_type_id != 9
        AND
        --Grabbing at least 2 years of data for comprehensive coverage of all pros
        DATE_TRUNC('month', l.lead_status_datetime) >= '2023-01-01'

    UNION ALL

    SELECT
        fp.sp_id,
        DATE_TRUNC('month', fp.create_datetime) AS month
    FROM Z AS fp
    WHERE
        fp.presentation_type NOT IN ('ALD', 'AL CM', 'AL DBS', 'AL DBS Historical', 'AL TL', 'SP Invite')
        AND
        --Grabbing at least 2 years of data for comprehensive coverage of all pros
        DATE_TRUNC('month', fp.create_datetime) >= '2023-01-01'
),

active_months_base AS (
    SELECT
        sp_id,
        month
    FROM active_months_temp
    GROUP BY sp_id, month
),

approved_with_flag AS (
    SELECT
        a.sp_id,
        a.month AS approved_month,
        a.approved_flag,
        CASE WHEN b.month IS NOT NULL THEN 1 ELSE 0 END AS flag
    FROM sp_status_monthly a
    LEFT OUTER JOIN active_months_base b
    ON a.sp_id = b.sp_id AND a.month = b.month
)

select 
    count(distinct sp_id) from approved_with_flag where approved_month = '2023-01-01' and flag = 1;

--2023-01-01 active pros should be 132138

'''

df = query(q)

In [33]:
df

,COUNT(DISTINCT SP_ID)
0,132136


In [ ]:
<h1> Ad Pro Active Check </h1>

In [38]:
q = '''

---THIS IS THE DEFINITION FOR ACTIVE AD PROS FROM DAVID S. ----

-- ACTIVE PROS - ADS
-- ZW Version

-- Ads Pros (ENGAGED PROS)

WITH sp_daily_contacts AS ( -----summarizes contact DATA INTO contacts BY DAY/sp
SELECT serviceproviderid, cast(OCCURRENCETIMESTAMP as date) rundate, count(DISTINCT contactid) Contacts
FROM LANDING.AL40.ADVERTISING_CONTACT ac 
WHERE 
ac.REASONID = 1 --Only contacts that were applied to the Budget Register are considered monetized transactions
group by 1,2
)

, daily_active AS (
SELECT DISTINCT
dsd.CONTRACTITEMSERVICEPROVIDERID sp_id,
dsd.RUNDATE,
sum(dsd.sprevenuerunrate) RRR
FROM rpt.AL_PRODLAKE.DATAFOUNDATION_SPCONTRACTITEMHISTORYDETAILS_DAILYSNAPSHOT dsd 
WHERE   dsd.AdTypeName = 'Web Ad'
    AND dsd.sprevenuerunrate > 0
--ZW added the same filters on here that we have on acquired pros.  This removes pros who use our non-core products (pub, call center, keyword, HR/FS) exclusively
GROUP BY 1,2
)

, active_days AS (
SELECT D.*,
lag(rundate) ignore nulls OVER (PARTITION BY sp_id ORDER BY rundate) prev_active_day, --find LAST active date
min(rundate) OVER (PARTITION BY sp_id) First_Advertiser_date, --FIRST date sp appears IN data
datediff(DAY,prev_active_day,rundate) Days_since_active, --calc days BETWEEN this rundate (day of data) AND the LAST date the pro was active, will RETURN NULL FOR FIRST advertiser dates
CASE WHEN days_since_active IS NULL OR days_since_active > 365 THEN rundate ELSE null END Reset_date, ---generate dates WHERE the SP becomes active AFTER MORE than 365 days inactive
FROM daily_active D
)

, tenure_resets AS (
SELECT DISTINCT
sp_id, 
rundate Run_date,
RRR,
First_Advertiser_date as NEW_TENURE_START_DATE, --Sets the Cohort date to the original cohort date
datediff(DAY,new_tenure_start_date,rundate) Tenure
FROM active_days
)

, daily AS (
SELECT
SP_ID, --'AL 4.0 ID'
RUN_DATE, --'Day of the data'
NEW_TENURE_START_DATE, --'resets if there is a break in contract coverage, represented in this table as revenue run rate (rrr) dropping to zero'
RRR, --'annualized value of the active ads contract'
sum(con.CONTACTs) Daily_contacts
FROM tenure_resets tr
LEFT JOIN sp_daily_contacts con ON con.SERVICEPROVIDERID = tr.SP_ID AND con.rundate = tr.RUN_DATE
WHERE RRR <> 0
GROUP BY 1,2,3,4
)

, pres as (
    SELECT cd.sp_id
        , cd.RUN_DATE AS engage_date
        , cd.NEW_TENURE_START_DATE as cohort_date
        , cd.RRR
        , cd.RUN_DATE
        , cd.NEW_TENURE_START_DATE
        , cd.Daily_contacts
        , count(distinct fp.presentation_id) Daily_Presentations
    FROM daily cd
    LEFT JOIN RPT.REPORTS.F_PRESENTATION fp
        ON cd.sp_id = fp.sp_id
        and coalesce(fp.sr_matched_datetime::date, fp.sr_submitted_date) = cd.run_date
        AND fp.presentation_type IN ('ALD','AL CM','AL DBS','AL DBS Historical','AL TL')
        AND fp.sr_submitted_date >= '1/1/2018'

    group by 1,2,3,4,5,6,7
) 

, activeadvertisers AS (
SELECT * FROM pres where (Daily_contacts > 0 OR Daily_Presentations > 0)  --Pros that received either a contact or a presentation are considered active

)
, monthly AS (
SELECT DISTINCT
sp_id,
CASE WHEN cohort_date < '2018-01-01' THEN '1900-01-01' ELSE DATE_TRUNC('month', cohort_date)::date END AS cohort_month,
CASE WHEN engage_date < '2018-01-01' THEN '1900-01-01' ELSE DATE_TRUNC('month', engage_date)::date END AS activity_month,
MAX(RRR) Max_RRR, --determining max RRR in the month to distinguish Large pros (>= $100K in max_RRR)
--ZW Changed the above line to MAX(RRR) to be consistent with Leads logic
sum(Daily_Contacts) Month_contacts, 
sum(Daily_Presentations) Month_Presentations,
count(DISTINCT run_date) Active_days
FROM activeadvertisers
GROUP BY ALL
)


, final AS (
SELECT
cohort_month,
activity_month,
count(DISTINCT CASE WHEN Max_RRR < 100000 THEN sp_id ELSE NULL END) non_large_ads_paying_pros,
count(DISTINCT CASE WHEN Max_RRR >= 100000 THEN sp_id ELSE NULL END) large_ads_paying_pros
FROM monthly
GROUP BY ALL
ORDER BY 1,2
)

, transformed_data AS (
    SELECT
        COHORT_MONTH,
        ACTIVITY_MONTH,
        'Large Ads' AS TYPE,
        'ACTIVE PROS' AS METRIC,
        'DUPE INCLUDED' AS COUNT_METHOD,
        LARGE_ADS_PAYING_PROS AS PRO_COUNT
    FROM final
    UNION ALL
    SELECT
        COHORT_MONTH,
        ACTIVITY_MONTH,
        'Non-Large Ads' AS TYPE,
        'ACTIVE PROS' AS METRIC,
        'DUPE INCLUDED' AS COUNT_METHOD,
        NON_LARGE_ADS_PAYING_PROS AS PRO_COUNT
    FROM final
),

MAIN AS (
SELECT
    COHORT_MONTH,
    ACTIVITY_MONTH,
    TYPE,
    METRIC,
    COUNT_METHOD,
    PRO_COUNT
FROM transformed_data
WHERE cohort_month is not null
    AND activity_month is not null
    AND activity_month >= cohort_month
ORDER BY
    COHORT_MONTH,
    ACTIVITY_MONTH,
    TYPE),

--new f_presentation_logic 


request_extension as (
SELECT
        oid AS sr_id,
        submitted_date AS sr_submitted_dttm,
        cast(trunc(submitted_date, 'DAY') AS TIMESTAMP_NTZ(9)) AS sr_submitted_date,
        marketplace_id,
        net_sr,
        alternate_source,
        matched_datetime,
        request_status,
        filter_status,
        match_type_id
    FROM RPT.TEMP.WK_BV_WS_SERVICE_REQUESTS_EXTENSION
    WHERE
        1 = 1
        AND test_sr = 'N'
        AND deleted = 0
        --Grabbing at least 2.5 years of data for comprehensive coverage of all pros
        AND modify_datetime >= '2025-06-01'
),

al_cm as (
SELECT
        cast(al.sr_al_pro_id AS STRING) || 'al_cm' || cast(al.al_pro_id AS STRING) AS presentation_id,
        'AL CM' AS presentation_type,  -- angies list combined match/project one
        al.sr_oid AS sr_id,
        al.al_pro_id AS sp_id, --equivalent to a sp_id
        al.create_datetime
    FROM landing.wisen_data.sm_sr_al_pro al
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        al.create_datetime >= '2025-06-01'
        and 
        al_pro_status_id IN (8, 9)
        AND NOT EXISTS (  -- <--- NOT EXISTS clause for 'AL CM'
            SELECT 1
            FROM rpt.intermediate.int_pro_selection_presented fep
            WHERE
                fep.sr_id = al.sr_oid
                AND fep.sp_id = al.al_pro_id
                AND fep.presentation_type = 'AL CM'
        )
    QUALIFY row_number() OVER (
        PARTITION BY al.sr_al_pro_id
        ORDER BY al.create_datetime
    ) = 1
),

al_dbs_h as (
SELECT
        cast(spr.sr_presentation_result_id AS STRING) || 'al_dbs_hist' || cast(spr.sp_id AS STRING) AS presentation_id,
        'AL DBS Historical' AS presentation_type,
        ssp.sr_oid AS sr_id,
        spr.sp_id,
        ssp.create_datetime
    FROM landing.wisen_data.sm_sr_presentation ssp
    INNER JOIN landing.wisen_data.sm_sr_presentation_result spr ON ssp.sr_presentation_id = spr.sr_presentation_id
    INNER JOIN request_extension fsr ON ssp.sr_oid = fsr.sr_id
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        ssp.create_datetime >= '2025-06-01'
        and 
        fsr.match_type_id IN (43, 9, 1, 3, 10, 51, 52, 46, 48)
        AND spr.marketplace_id = 2
        AND spr.sr_prsntn_rslt_type_id IN (14)
        ---and spr.is_phone_displayed is not null
        AND (fsr.net_sr = 1 OR fsr.request_status = 'EXT_FULFIL')
),

al_dbs as (
SELECT
        cast(ap.sr_al_pro_id AS STRING) || 'al_dbs' || cast(ap.al_pro_id AS STRING) AS presentation_id,
        'AL DBS' AS presentation_type,
        ap.sr_oid AS sr_id,
        ap.al_pro_id AS sp_id,
        ap.create_datetime
    FROM landing.wisen_data.sm_sr_al_pro ap
    WHERE --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        ap.create_datetime >= '2025-06-01'
        and ap.al_pro_status_id IN (2, 3, 4)
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY ap.create_datetime
        ) = 1
),

al_tl as (
SELECT
        cast(ap.sr_al_pro_id AS STRING) || 'al_tl' || cast(ap.al_pro_id AS STRING) AS presentation_id,
        'AL TL' AS presentation_type,
        ap.sr_oid AS sr_id,
        ap.al_pro_id AS sp_id,
        ap.create_datetime
    FROM landing.wisen_data.sm_sr_al_pro ap
    WHERE --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        ap.create_datetime >= '2025-06-01'
        and 
        ap.al_pro_status_id IN (5, 6)
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY ap.create_datetime
        ) = 1
),

ald as (
SELECT
        cast(spr.sr_presentation_result_id AS STRING) || 'ald' || cast(spr.sp_id AS STRING) AS presentation_id,
        'ALD' AS presentation_type,
        ssp.sr_oid AS sr_id,
        spr.sp_id,
        ssp.create_datetime
    FROM landing.wisen_data.sm_sr_presentation ssp
    INNER JOIN landing.wisen_data.sm_sr_presentation_result spr ON ssp.sr_presentation_id = spr.sr_presentation_id
    INNER JOIN request_extension fsr ON ssp.sr_oid = fsr.sr_id
    LEFT OUTER JOIN landing.wisen_data.sm_sr_al_pro al ON ssp.sr_oid = al.sr_oid
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        ssp.create_datetime >= '2025-06-01'
        and 
        fsr.match_type_id IN (43, 9, 1, 3, 10, 51, 52, 46, 48)
        AND spr.marketplace_id = 2
        AND spr.sr_prsntn_rslt_type_id NOT IN (14, 16)
        --and spr.is_phone_displayed is not null
        AND (fsr.net_sr = 1 OR fsr.request_status = 'EXT_FULFIL')
        AND al.sr_oid IS NULL
),

basecc as (
        SELECT
        choice_pot_lead_id,
        sr_oid as sr_id,
        sp_id,
        min(create_datetime) AS min_date
    FROM
        landing.wisen_data.sm_choice_pot_lead_hist
    WHERE
        choice_pot_lead_status_id IN (2, 4)
        AND post_submit_match_fg = 'N'
        AND presented_datetime IS NOT NULL
        AND --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        create_datetime >= '2025-06-01'
    GROUP BY choice_pot_lead_id, sr_oid, sp_id
    ),
        
cc as (
SELECT
        cast(cc.choice_pot_lead_id AS STRING) || 'cc' || cast(cc.sp_id AS STRING) AS presentation_id,
        'Consumer Choice' AS presentation_type,
        cc.sr_id,
        cc.sp_id,
        cc.min_date AS create_datetime
    FROM basecc cc
    INNER JOIN request_extension fsr ON cc.sr_id = fsr.sr_id
    WHERE
        (fsr.net_sr = 1 OR fsr.request_status = 'EXT_FULFIL')
        AND cast(fsr.sr_submitted_dttm AS DATE) BETWEEN cast('2019-09-30' AS DATE) AND current_timestamp
        AND NOT EXISTS (  -- <--- NOT EXISTS clause for 'Consumer Choice'
            SELECT 1
            FROM rpt.intermediate.int_pro_selection_presented fep
            WHERE
                fep.sr_id = cc.sr_id
                AND fep.sp_id = cc.sp_id
                AND fep.presentation_type = 'Consumer Choice'
        )

    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY cc.min_date
        ) = 1
),

cj as (
SELECT
        cast(cj.sr_cj_pro_id AS STRING) || 'cj' || cast(cj.sp_id AS STRING) AS presentation_id,
        'CJ' AS presentation_type,  -- angies list combined match/project one
        cj.sr_oid AS sr_id,
        cj.sp_id, --equivalent to a sp_id
        cj.create_datetime
    FROM landing.wisen_data.SM_SR_CJ_PRO cj
    WHERE --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        cj.create_datetime >= '2025-06-01'
        and 
        cj_pro_status_id IN (1, 2)
),

srs_with_ol_pros_presented AS (
    SELECT sr_oid as sr_id
    FROM rpt.intermediate.int_ol

    UNION DISTINCT

    SELECT sr_id
    FROM rpt.intermediate.int_pro_selection_presented
),

fpp as (
select 
        cast(eft.ext_fulfillment_trans_id AS STRING) || 'fpp' || '0' AS presentation_id,
        'Fixed Price Path' AS presentation_type,
        eft.sr_oid AS sr_id,
        NULL AS sp_id,
        eft.create_datetime
    FROM landing.wisen_data.SM_EXT_FULFILLMENT_TRANS eft
    INNER JOIN request_extension fsr
    ON eft.sr_oid = fsr.sr_id
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        eft.create_datetime >= '2025-06-01'
        and 
        (
        --2025-06-11 is the first full day when FPP traffic was migrated to 
        -- the new frontend in next-sr-path. A Service Request that happend 
        -- after this date and has all of: 
        --  1. An OL presentation 
        --  2. Match type 56 
        --  3. Alternate source SITE or AL_SITE
        --  Should not be considered an FPP presentation.
            eft.create_datetime < '2025-06-11'
            OR
            (
                eft.create_datetime >= '2025-06-11'
                AND (
                    fsr.match_type_id NOT IN (56)
                    OR fsr.alternate_source NOT IN (1, 14)
                    OR NOT EXISTS (
                        SELECT 1
                        FROM srs_with_ol_pros_presented olp
                        WHERE olp.sr_id = eft.sr_oid
                    )
                )
            )
        )
    QUALIFY
        row_number() OVER (
            PARTITION BY eft.sr_oid
            ORDER BY eft.create_datetime
        ) = 1
),

ib as (
SELECT
        cast(ib.ob_potential_lead_id AS STRING) || 'ib' || cast(ib.sp_id AS STRING) AS presentation_id,
        'Instant Booking' AS presentation_type,
        ib.sr_oid AS sr_id,
        ib.sp_id,
        ib.create_datetime
    FROM landing.wisen_data.SM_OB_POTENTIAL_LEAD ib
    INNER JOIN request_extension fsr ON ib.sr_oid = fsr.sr_id
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        ib.create_datetime >= '2025-06-01'
        and 
        fsr.match_type_id IN (25, 26, 56)
        AND ib.display_seq > 0
        AND NOT EXISTS (  -- <--- NOT EXISTS clause for 'Instant Booking'
            SELECT 1
            FROM rpt.intermediate.int_pro_selection_presented fep
            WHERE
                fep.sr_id = ib.sr_oid
                AND fep.sp_id = ib.sp_id
                AND fep.presentation_type = 'Instant Booking'
        )
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY ib.create_datetime
        ) = 1
),

ic as (
SELECT
        cast(ic.ic_pot_lead_id AS STRING) || 'ic' || cast(ic.sp_id AS STRING) AS presentation_id,
        'Instant Connect' AS presentation_type,
        ic.sr_oid AS sr_id,
        ic.sp_id,
        ic.create_datetime
    FROM landing.wisen_data.SM_INSTANT_CONNECT_POT_LEAD ic
    WHERE --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        ic.create_datetime >= '2025-06-01'
        and 
        ic.status_id IN (3) --removed 1 which is 'not called'. including 1 returned the pool of SPs, not just the one connected
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY ic.create_datetime
        ) = 1
),

mhc as (
SELECT
        cast(poh.prepriced_order_id AS STRING) || 'mhc' || '0' AS presentation_id,
        'MHC' AS presentation_type,
        poh.sr_oid AS sr_id,
        NULL AS sp_id,
        poh.create_datetime
    FROM landing.product_storefront.PREPRICED_ORDER_HIST poh
    WHERE 
    --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        poh.create_datetime >= '2025-06-01'
        and poh.sr_oid IS NOT NULL
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY poh.create_datetime
        ) = 1
),

mmr as (
SELECT
        cast(pm.map_id AS STRING) || 'mmr' || cast(pm.sp_id AS STRING) AS presentation_id,
        'MMR' AS presentation_type,
        pm.sr_oid AS sr_id,
        pm.sp_id,
        pm.create_datetime
    FROM landing.wisen_data.SM_XM_DIR_PHONE_MAP pm
    INNER JOIN request_extension fsr ON pm.sr_oid = fsr.sr_id
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        pm.create_datetime >= '2025-06-01'
        and 
        match_type_id IN (9, 10, 24, 56)
        AND (fsr.net_sr = 1 OR fsr.request_status = 'EXT_FULFIL')
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY pm.create_datetime
        ) = 1
),

mw as (
SELECT
        cast(fl.lead_id AS STRING) || 'mm' || cast(fl.sp_id AS STRING) AS presentation_id,
        'Market Match' AS presentation_type,
        fl.sr_oid as sr_id,
        fl.sp_id,
        fl.create_datetime
    FROM landing.wisen_data.sm_lead fl
    INNER JOIN REQUEST_EXTENSION fsr ON fl.sr_oid = fsr.sr_id
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        fl.create_datetime >= '2025-06-01'
        and 
        (fsr.net_sr = 1 OR fsr.request_status = 'EXT_FULFIL')
        AND fsr.match_type_id IN (1, 43)
        AND fl.lead_type_id IN (2)
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY fl.create_datetime
        ) = 1
),

mmr as (
SELECT
        cast(fl.lead_id AS STRING) || 'mm' || cast(fl.sp_id AS STRING) AS presentation_id,
        'Market Match' AS presentation_type,
        fl.sr_oid,
        fl.sp_id,
        fl.create_datetime
    FROM landing.wisen_data.sm_lead fl
    INNER JOIN request_extension fsr ON fl.sr_oid = fsr.sr_id
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        fl.create_datetime >= '2025-06-01'
        and 
        (fsr.net_sr = 1 OR fsr.request_status = 'EXT_FULFIL')
        AND fsr.match_type_id IN (1, 43)
        AND fl.lead_type_id IN (2)
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY fl.create_datetime
        ) = 1
),

ol_tl as (
SELECT
        cast(fol.ol_pot_lead_id AS STRING) || 'oltl' || cast(fol.sp_id AS STRING) AS presentation_id,
        'OL TL' AS presentation_type,
        fol.sr_oid as sr_id,
        fol.sp_id,
        fol.create_datetime
    FROM rpt.intermediate.int_ol fol
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        fol.create_datetime >= '2025-06-01'
        and 
        fol.ol_type_id != 2
        AND fol.ol_type_id != 3
        AND fol.sp_presented_on_tl = 1
        AND NOT EXISTS (  -- <--- NOT EXISTS clause for 'OL TL'
            SELECT 1
            FROM rpt.intermediate.int_pro_selection_presented fep
            WHERE
                fep.sr_id = fol.sr_oid
                AND fep.sp_id = fol.sp_id
                AND fep.presentation_type = 'OL TL'
        )
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY fol.create_datetime
        ) = 1
),

pf as (
SELECT
        cast(ppl.pmp_pot_lead_id AS STRING) || 'profinder' || cast(ppl.sp_id AS STRING) AS presentation_id,
        'Profinder' AS presentation_type,
        ppl.sr_oid AS sr_id,
        ppl.sp_id,
        ppl.create_datetime
    FROM landing.wisen_data.SM_PMP_POT_LEAD ppl
    WHERE --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        ppl.create_datetime >= '2025-06-01'
        and 
        pmp_pot_lead_status_id IN (2, 4)
),

ping_post as (
SELECT
        bid_pro.bidproid,
        bid_pro.id_bidkey,
        bid_pro.id_legacyserviceproviderid,
        bid_service_request.servicerequestid,
        bid_pro.createtime_ntz,
        bid_pro.met
    FROM
        landing.kafka.ppp_data_bid_pro AS bid_pro
    INNER JOIN landing.kafka.PPP_DATA_BID_SERVICE_REQUEST AS bid_service_request
    ON
        bid_pro.id_bidkey = bid_service_request.bidkey
    WHERE
        bid_pro.met in ('XML_OL','XML_SR_BID')
    and 
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        bid_pro.createtime_ntz >= '2025-06-01'
),

ppp_ol as (
    SELECT
        bidproid || 'pppol' || id_legacyserviceproviderid AS presentation_id,
        'ping-post-pick OL' AS presentation_type,
        servicerequestid AS sr_id,
        id_legacyserviceproviderid AS sp_id,
        createtime_ntz AS create_datetime
    FROM
        ping_post AS bid_pro
    WHERE 
        met = 'XML_OL'
    QUALIFY
        row_number() OVER (
            PARTITION BY 
                CAST(bidproid AS VARCHAR) || 'pppol' || CAST(id_legacyserviceproviderid AS VARCHAR)
            ORDER BY createtime_ntz DESC
        ) = 1
),

ppp as (
SELECT
        bidproid || 'ppp' || id_legacyserviceproviderid AS presentation_id,
        'ping-post-pick' AS presentation_type,
        servicerequestid AS sr_id,
        id_legacyserviceproviderid AS sp_id,
        createtime_ntz AS create_datetime
    FROM
        ping_post
    WHERE
        met = 'XML_SR_BID'
    QUALIFY
        row_number() OVER (
            PARTITION BY bidproid || 'ppp' || id_legacyserviceproviderid
            ORDER BY createtime_ntz DESC
        ) = 1
),

psm as (
SELECT
        cast(cplh.choice_pot_lead_id AS STRING) || 'psm' || cast(cplh.sp_id AS STRING) AS presentation_id,
        'PSM' AS presentation_type,
        cplh.sr_oid AS sr_id,
        cplh.sp_id,
        cplh.create_datetime
    FROM landing.wisen_data.SM_CHOICE_POT_LEAD_HIST cplh
    INNER JOIN request_extension fsr ON cplh.sr_oid = fsr.sr_id
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        cplh.create_datetime >= '2025-06-01'
        and 
        cplh.post_submit_match_fg = 'Y'
        AND choice_pot_lead_status_id IN (2)
        AND cplh.presented_datetime IS NOT NULL
        AND (fsr.net_sr = 1 OR fsr.request_status = 'EXT_FULFIL')
        AND fsr.sr_submitted_dttm < current_timestamp
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY cplh.create_datetime
        ) = 1
),

sd as (
SELECT
        cast(sdb.sdb_potential_lead_id AS STRING) || 'sdb' || cast(sdb.sp_id AS STRING) AS presentation_id,
        'Same Day Booking' AS presentation_type,
        sdb.sr_oid AS sr_id,
        sdb.sp_id,
        sdb.create_datetime
    FROM landing.wisen_data.SM_SDB_POTENTIAL_LEAD sdb
    where
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        sdb.create_datetime >= '2025-06-01'
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY sdb.create_datetime
        ) = 1
),

spi as (
SELECT
        cast(fol.ol_pot_lead_id AS STRING) || 'spi' || cast(fol.sp_id AS STRING) AS presentation_id,
        'SP Invite' AS presentation_type,
        fol.sr_oid as sr_id,
        fol.sp_id,
        fol.create_datetime
    FROM rpt.intermediate.int_ol fol
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        fol.create_datetime >= '2025-06-01'
        and
        fol.sp_invites = 1
        AND fol.sp_presented_on_tl = 0
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY fol.create_datetime
        ) = 1
),

wj as (
SELECT
        cast(fol.ol_pot_lead_id AS STRING) || 'winjobs' || cast(fol.sp_id AS STRING) AS presentation_id,
        'Win Jobs' AS presentation_type,
        fol.sr_oid as sr_id,
        fol.sp_id,
        fol.create_datetime
    FROM rpt.intermediate.int_ol fol
    WHERE
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        fol.create_datetime >= '2025-06-01'
        AND 
            fol.ol_type_id = 3
        AND fol.sp_presented_on_tl = 1
    QUALIFY
        row_number() OVER (
            PARTITION BY presentation_id
            ORDER BY fol.create_datetime
        ) = 1
),

final_prez as (
SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        datetime_mst AS create_datetime
    FROM rpt.intermediate.int_pro_selection_presented
    WHERE 
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        datetime_mst >= '2025-06-01'

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM cc

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM ib

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM ic

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM fpp

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM mmr

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM ppp

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM ppp_ol

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM sd

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM mhc

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM ol_tl

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM spi

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM wj


    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM psm

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM ald

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM al_tl

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM al_dbs

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM al_dbs_h

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM mw

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM pf

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM cj

    UNION ALL

    SELECT
        presentation_id,
        presentation_type,
        sr_id,
        sp_id,
        create_datetime
    FROM al_cm
),

base_migrated as (
    select 
        sr_id,
        sp_id,
        presentation_type,
        create_datetime 
    from 
        final_prez
    where 
        --Grabbing only 3 years of historical data for YoY comparisons in rpt.reports.f_migration_flags
        create_datetime >= '2025-06-01'
    ),
    
    double_counts_ib_cc AS (
    SELECT
        sr_id,
        sp_id,
        'Instant Booking' AS presentation_type
    FROM base_migrated
    WHERE presentation_type IN ('Consumer Choice', 'Instant Booking')
    GROUP BY 1, 2
    HAVING count(*) > 1
    ),
    
Z AS (
SELECT
    b.sp_id,
    b.create_datetime,
    fsr.sr_submitted_date,
    fsr.matched_datetime AS sr_matched_datetime,
    b.presentation_type
FROM base_migrated b
INNER JOIN request_extension  fsr ON b.sr_id = fsr.sr_id
LEFT OUTER JOIN landing.wisen_data.sm_lead sl ON b.sr_id = sl.sr_oid AND b.sp_id = sl.sp_id
LEFT OUTER JOIN double_counts_ib_cc db ON (b.sr_id = db.sr_id AND b.sp_id = db.sp_id)
WHERE
    (sl.lead_type_id != 9 OR sl.lead_type_id IS NULL)
    AND (db.presentation_type IS NULL OR db.presentation_type = b.presentation_type)
),

z_final as (
select 
    date(create_datetime) as date,
    count(distinct sp_id) as pros 
from 
    Z 
group by 1),

 --------
 
 base2 AS (
    SELECT
        rundate,
        contractitemserviceproviderid AS serviceproviderid
    FROM rpt.al_prodlake.datafoundation_spcontractitemhistorydetails_dailysnapshot
    WHERE
        adtypename = 'Web Ad'
        AND sprevenuerunrate > 0
        --Grabbing at least 2 years of data for comprehensive coverage of all pros 
        AND rundate >= '2023-01-01'
),

sp_daily_contacts2 AS (
    SELECT
        serviceproviderid,
        CAST(occurrencetimestamp AS DATE) AS contact_date
    FROM landing.al40.advertising_contact
    WHERE
        reasonid = 1
        AND
        --Grabbing at least 2 years of data for comprehensive coverage of all pros
        DATE(occurrencetimestamp) >= '2023-01-01'
),

presentations AS (
    SELECT
        sp_id AS serviceproviderid,
        COALESCE(CAST(sr_matched_datetime AS DATE), CAST(sr_submitted_date AS DATE)) AS presentation_date
    FROM z
    WHERE
        presentation_type IN ('ALD', 'AL CM', 'AL DBS', 'AL DBS Historical', 'AL TL')
        AND
        --Grabbing at least 2 years of data for comprehensive coverage of all pros
        sr_submitted_date >= '2023-01-01'
),

-- Flag active days
active_flagged_daily AS (
    SELECT
        da.serviceproviderid,
        da.rundate,
        DATE_TRUNC('month', da.rundate) AS activity_month,
        CASE
            WHEN c.serviceproviderid IS NOT NULL OR p.serviceproviderid IS NOT NULL THEN 1
            ELSE 0
        END AS is_active
    FROM base2 da
    LEFT OUTER JOIN sp_daily_contacts2 c
    ON da.serviceproviderid = c.serviceproviderid AND da.rundate = c.contact_date
    LEFT OUTER JOIN presentations p
    ON da.serviceproviderid = p.serviceproviderid AND da.rundate = p.presentation_date
    WHERE da.serviceproviderid IS NOT NULL
),

-- Collapse to one flag per month
monthly_flags AS (
    SELECT
        serviceproviderid,
        activity_month,
        MAX(is_active) AS flag  -- if any day was active in the month, the month is flagged 1
    FROM active_flagged_daily
    GROUP BY serviceproviderid, activity_month
),

---Comparison Check

zakir as (
select 
    activity_month,
    count(distinct serviceproviderid) as zakir_pros
from 
    monthly_flags 
where 
    flag = 1
and 
    activity_month >= '2023-01-01'
group by 1
),


david as (
select 
    activity_month,
    sum(pro_count) as david_pros
from 
    main 
where 
activity_month >= '2023-01-01'
group by 1)


select 
    a.activity_month,
    zakir_pros,
    david_pros,--Percent are similiar for 2023-01-01?
    round(zakir_pros/david_pros,2)*100
from 
    david a 
join 
    zakir b 
on a.activity_month = b.activity_month

'''

df_ads = query(q)

In [39]:
df_ads

,ACTIVITY_MONTH,ZAKIR_PROS,DAVID_PROS,"ROUND(ZAKIR_PROS/DAVID_PROS,2)*100"
0,2023-01-01,37527,37751,99.0
1,2025-02-01,35050,35382,99.0
2,2023-10-01,41311,41568,99.0
3,2023-08-01,40812,40983,100.0
4,2024-01-01,39907,40282,99.0
5,2023-12-01,40216,40671,99.0
6,2025-06-01,27752,27765,100.0
7,2023-04-01,38294,38415,100.0
8,2025-03-01,33442,33578,100.0
9,2023-09-01,41038,41300,99.0
